In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import os
import cv2

In [2]:
#x,y,c,
poseName = 'Retire'
Coords = np.load("RootCoordinates/"+poseName+"X.npy")
print(Coords)
print(len(Coords))

[[0.489774 0.284766 0.916418 ... 0.520393 0.739914 0.842526]
 [0.493769 0.284897 0.893677 ... 0.520352 0.742638 0.865259]
 [0.489759 0.28487  0.896611 ... 0.520349 0.742721 0.86492 ]
 ...
 [0.481682 0.30923  0.898575 ... 0.530618 0.747905 0.795508]
 [0.481658 0.303868 0.898717 ... 0.53067  0.747903 0.822877]
 [0.483563 0.306545 0.899593 ... 0.532487 0.745286 0.793008]]
870


In [3]:
#Helper Methods:
import math
def calcDistance(x1,y1,x2,y2):
    distance = math.sqrt((x2-x1)**2 + (y2-y1)**2)
    return distance

In [4]:
#Get the Mean Skeleton's coordinates
PoseMeanX = np.load("Stats/"+poseName+"MeanX.npy")
print(PoseMeanX)
print(len(PoseMeanX))
PoseMeanY = np.load("Stats/"+poseName+"MeanY.npy")
print(PoseMeanY)
print(len(PoseMeanY))
MeanTuples = list(zip(PoseMeanX, PoseMeanY))
print(MeanTuples)

[0.48401868 0.48401022 0.45132468 0.40990527 0.44111402 0.51628598
 0.5581041  0.52767739 0.5        0.47951456 0.50040566 0.51543082
 0.52016344 0.59563769 0.53365008 0.47682349 0.4911798  0.4672444
 0.50129298 0.52320113 0.51813933 0.52261083 0.49035051 0.48900637
 0.52208075]
25
[-0.30068264 -0.36697015 -0.36778491 -0.40743698 -0.42638773 -0.36562421
 -0.40490714 -0.42309388 -0.5        -0.5090083  -0.62131171 -0.73525073
 -0.49162485 -0.51985107 -0.59105991 -0.2927469  -0.29259385 -0.30599397
 -0.30520785 -0.61966363 -0.61265782 -0.5911934  -0.76005237 -0.75349693
 -0.74637757]
25
[(0.48401868045977003, -0.30068263563218406), (0.48401021839080466, -0.3669701482758617), (0.4513246758620693, -0.36778491494252874), (0.40990526666666643, -0.4074369770114938), (0.44111401954023083, -0.42638773103448274), (0.5162859816091953, -0.36562421264367806), (0.5581041034482758, -0.404907142528736), (0.5276773896551725, -0.42309387586206926), (0.5, -0.5), (0.47951456321839064, -0.509008301149425),

In [5]:
POSE = "Retire"
allDistanceVectors = []
for sample in Coords:
    xCoords = sample[0::3]
    #print(xCoords)
    yPos = sample[1::3]
    yCoords = [-y for y in yPos] # need to make it negative to be consistent - the mean skeleton's y saved the negative coords.
    SampleTuples = list(zip(xCoords, yCoords))
    #print(SampleTuples)
    longZip = list(zip(MeanTuples, SampleTuples))
    #print(longZip)
    distanceVector = []
    for ((x1,y1),(x2,y2)) in longZip:
        d = calcDistance(x1,y1,x2,y2)
        #print(d)
        distanceVector.append(d)
    allDistanceVectors.append(distanceVector)

print(allDistanceVectors)
print(len(allDistanceVectors))
print(len(allDistanceVectors[0]))
np.save("DataForSVM/"+POSE+"DistanceVectorsPerSample.npy", allDistanceVectors)

[[0.01692521766057571, 0.016689444995579443, 0.017820002884905622, 0.030172786825108676, 0.02716867891011636, 0.015359667078813562, 0.016806446851605934, 0.026589002989601672, 0.0, 0.006076367690409374, 0.006756905422216571, 0.008995424497665768, 0.005417935088822149, 0.003063506976092696, 0.007169475857386388, 0.015031979085006841, 0.0158430928344532, 0.01890030746031973, 0.018202783051978547, 0.010746200521294451, 0.014462984646251812, 0.010106509795263351, 0.003703837575448773, 0.00849730933258877, 0.006680285242088944], [0.018554110688703662, 0.014406521607213006, 0.015208476008640738, 0.025286035709033497, 0.024456746155490573, 0.013195666599723986, 0.012381322040573994, 0.021289208274948842, 0.0, 0.004030379489402154, 0.004358788217631373, 0.0036661410130660083, 0.005523547173132228, 0.005206108901446298, 0.0032459541495392283, 0.01385409750502535, 0.013766349004718794, 0.017663858717156648, 0.017329244720478382, 0.007708757322083008, 0.015097568133000276, 0.004110545493672224, 0

In [6]:
#These features are mixing coordinates and distances: 100 features per sample: 75 + 25
PoseAugFeaturesX = []
for i in range(len(Coords)):
    newList = list(Coords[i])+list(allDistanceVectors[i])
    PoseAugFeaturesX.append(newList)
    
print(PoseAugFeaturesX[0])
print(len(PoseAugFeaturesX[0]))
np.save("DataForSVM/"+POSE+"AugMeanDistFeaturesX.npy", PoseAugFeaturesX)

[0.48977400000000004, 0.284766, 0.916418, 0.48576300000000006, 0.35037300000000005, 0.9253619999999999, 0.4550140000000001, 0.350351, 0.908129, 0.41421800000000003, 0.377574, 0.835421, 0.438717, 0.39932500000000004, 0.8403189999999999, 0.518402, 0.35041100000000003, 0.8925679999999999, 0.5612670000000001, 0.38840100000000005, 0.8581200000000001, 0.530645, 0.39667100000000005, 0.8501029999999999, 0.5, 0.5, 0.835943, 0.47954300000000005, 0.502932, 0.761953, 0.500073, 0.614563, 0.851688, 0.516275, 0.726295, 0.8383, 0.518459, 0.48648199999999997, 0.7768889999999999, 0.597983, 0.521822, 0.881907, 0.536718, 0.58458, 0.815044, 0.4837, 0.27938, 0.931601, 0.49994000000000005, 0.27939300000000006, 0.929974, 0.47143900000000005, 0.287565, 0.876968, 0.5061100000000001, 0.287654, 0.83789, 0.530567, 0.611839, 0.571547, 0.532579, 0.6118370000000001, 0.570439, 0.526502, 0.581866, 0.543461, 0.493877, 0.75892, 0.727109, 0.491777, 0.745464, 0.72148, 0.520393, 0.739914, 0.842526, 0.01692521766057571, 0.01